In [2]:
%config ZMQInteractiveShell.ast_node_interactivity='all'
%matplotlib inline
import warnings;warnings.filterwarnings('ignore')
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn import metrics
from sklearn.base import BaseEstimator, ClassifierMixin, TransformerMixin, clone
from sklearn.preprocessing import Imputer, LabelEncoder
from sklearn.model_selection import KFold, StratifiedKFold, RandomizedSearchCV
from sklearn.externals import joblib

from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegressionCV
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

/Users/Meng/opt/anaconda3/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [3]:
target = '标签'
uid = '申请编号'

def calc_auc(y_test, y_proba):
    auc = round(metrics.roc_auc_score(y_test, y_proba), 3)
    return auc

def ks_score(y_test, y_proba):
    scale = 4
    fpr, tpr, thresholds = metrics.roc_curve(y_test, y_proba, pos_label=1)
    KS = round(max(list(tpr-fpr)), scale)
    return KS

In [4]:
############## Load

X = pd.read_csv('./tmp/train_d12_d3_dum.csv', header=0, index_col=0).drop(uid, axis=1).values
# X = pd.read_csv('./tmp/1_X.csv', header=0, index_col=0).values
len(X)
len(X[0])

y = pd.read_csv('./tmp/1_y.csv', header=0, index_col=0)['0'].values
len(y)

140000

390

140000

In [19]:
# For kfold
n_folds = 2

# RF
param_fixed_rf = {
    'n_jobs' : -1,
    'oob_score' : True,
    'random_state':123,
    'verbose':0
}

# XGB
param_fixed_xgb = {
    'n_jobs' : -1,
    'eval_metric': 'auc',
    'seed' : 123,
    'silent' : 1,
    'verbose_eval':0
}

# LGB
param_fixed_lgb = {
    'n_jobs' : -1,
    'metric' : 'auc',
    'bagging_seed':123,
    'feature_fraction_seed':123,
    'verbose_eval' : 0
}

# LR
param_fixed_lr = {
#     'cv' : cv,  # LRCV
    'solver' : 'sag',
    'n_jobs' : -1,
    'random_state' : 123,
    'verbose' : 0     
}

In [ ]:
class StackingModels (BaseEstimator, ClassifierMixin, TransformerMixin):
    def __init__(self, base_models, meta_model, n_folds = n_folds):
        self.base_models = base_models
        self.meta_model = meta_model
        self.n_folds = n_folds
        self.meta_X = []

    def fit(self, X, y):
        self.base_models_ = [list() for x in self.base_models]
        kfold = KFold(n_splits=self.n_folds, shuffle=True, random_state=123)
        
        # Get results of basic models
        out_of_fold_predictions = np.zeros((X.shape[0], len(self.base_models)))
        for i, model in enumerate(self.base_models):
            print(f'*** Base Model: {model.__class__} ***')
            j = 0
            for train_index, valid_index in kfold.split(X, y):
                instance = clone(model)
                self.base_models_[i].append(instance)
                instance.fit(X[train_index],  y[train_index])
                y_pred = instance.predict_proba(X[valid_index])[:,1]
#                     y_pred = instance.predict(X[valid_index])
                ks = ks_score(y[valid_index], y_pred)
                print(f'* KS({j}):{ks} *')
                out_of_fold_predictions[valid_index, i] = y_pred
                j += 1

        # META
        self.meta_X = out_of_fold_predictions
        self.meta_model_ = clone(self.meta_model)
        self.meta_model_.fit(out_of_fold_predictions, y)
        return self

    # 从得到的新的特征  采用新的模型进行预测  并输出结果
    def predict(self, X, if_meta=True):
        meta_features = np.column_stack ([
            np.column_stack(
#                 [model.predict(X) for model in base_models]
                [model.predict_proba(X)[:,1] for model in base_models]
            ).mean(axis=1)
            for base_models in self.base_models_
        ])
        if if_meta:
            pred = self.meta_model_.predict_proba(meta_features)[:,1]
        else:
            pred = meta_features.mean(axis=1)
        return pred

############ Stacking

# RF = RandomForestClassifier()
# XGB = XGBClassifier()
# LGB = LGBMClassifier()
# LR = LogisticRegression()

# RF
best_params_load = np.load('./model/base_rf.npy', allow_pickle=True).item()
model_params = {**best_params_load, **param_fixed_rf}
RF = RandomForestClassifier(**model_params)
# XGB
best_params_load = np.load('./model/base_xgb.npy', allow_pickle=True).item()
model_params = {**best_params_load, **param_fixed_xgb}
XGB = XGBClassifier(**model_params)
# LGB
best_params_load = np.load('./model/base_lgb.npy', allow_pickle=True).item()
model_params = {**best_params_load, **param_fixed_lgb}
LGB = LGBMClassifier(**model_params)
# LR
best_params_load = np.load('./model/base_lr.npy', allow_pickle=True).item()
model_params = {**best_params_load, **param_fixed_lr}
LR = LogisticRegression(**model_params)
# Mean
MEAN = 'MEAN'

### Stacking
CLF = StackingModels(base_models=[LGB, RF, XGB], meta_model=LR)

# Fit
CLF.fit(X, y)

# Predict
pred = CLF.predict(X, if_meta=False)
ks_score(y, pred)

# # Save meta_X
# meta_X = pd.DataFrame(CLF.meta_X, columns=['lgb', 'rf', 'xgb'])
# meta_X.shape
# meta_X.head()
# meta_X.to_csv('./model/x_meta.csv')

*** Base Model: <class 'lightgbm.sklearn.LGBMClassifier'> ***
* KS(0):0.3336 *


In [22]:
############## Test

def norm_score(score):
    if score < 0:
        score = 0
    elif score > 1:
        score = 1
    return score

test = pd.read_csv('./tmp/A_d12_d3_dum.csv', header=0, index_col=0)
X_test = test.drop(uid, axis=1).values
# X = pd.read_csv('./tmp/1_X.csv', header=0, index_col=0).values
len(X_test)
len(X_test[0])

# Predict
scores = CLF.predict(X_test)

# Output
uids = test[uid]
out = pd.concat([uids, pd.Series(scores)], axis=1)
out[0] = out[0].apply(norm_score)
out.head()
out.to_csv('./model/predict.csv', header=False, index=False)

21511

390

,申请编号,0
0,122687,0.074004
1,32425,0.070484
2,2024,0.163971
3,25019,0.203157
4,162532,0.088576


In [ ]:
############## Ana

base_models = [LGB, RF, LR, XGB]
base_models_ = [list() for x in base_models]
kfold = KFold(n_splits=2, shuffle=True, random_state=123)

out_of_fold_predictions = np.zeros((X.shape[0], len(base_models)))
for i, model in enumerate(base_models):
    j = 0
    for train_index, valid_index in kfold.split(X, y):
        instance = clone(model)
        base_models_[i].append(instance)
        instance.fit(X[train_index],  y[train_index])
        len(X[train_index])
#         y_pred = instance.predict(X[valid_index])
        y_pred = instance.predict_proba(X[valid_index])[:,1]
        ks = ks_score(y[valid_index], y_pred)
        print(f'* KS({j}):{ks} *')
        out_of_fold_predictions[valid_index, i] = y_pred
        j += 1

In [ ]:
len(out_of_fold_predictions)
len(out_of_fold_predictions[0])
out_of_fold_predictions[0]

In [ ]:
meta_model = LR
meta_model_ = clone(meta_model)

meta_model_.fit(out_of_fold_predictions, y)
y_pred = meta_model_.predict_proba(out_of_fold_predictions)[:,1]
ks_score(y, y_pred)

In [ ]:
meta_features = np.column_stack ([
    np.column_stack(
        [model.predict_proba(X)[:,1] for model in base_models]
    ).mean (axis=1)
    for base_models in base_models_])

In [ ]:
len(meta_features)
len(meta_features[0])

In [ ]:
''' RF '''
RF.fit(meta_features, y)
ks_score(y, RF.predict_proba(meta_features)[:,1])

''' RF1 '''
RF1 = RandomForestClassifier()
RF1.fit(meta_features, y)
ks_score(y, RF1.predict_proba(meta_features)[:,1])

''' LGB '''
LGB.fit(meta_features, y)
ks_score(y, LGB.predict_proba(meta_features)[:,1])

''' LR '''
LR.fit(meta_features, y)
ks_score(y, LR.predict_proba(meta_features)[:,1])

''' MEAN '''
# LR.fit(meta_features, y)
ks_score(y, meta_features.mean(axis=1))